# Validate Binding Isotherm

#### Import Modules

In [1]:
import os
import sys

cwd = os.getcwd()
parent_dir = cwd + "/../.."
sys.path.insert(1, parent_dir)

print("Directory containing the notebook:")
print(cwd)

Directory containing the notebook:
/Users/jwakim/Documents/WS_crosstalk_2024/chromo/simulations/validations


In [2]:
os.chdir(parent_dir)
print("Root Directory of Package: ")
print(os.getcwd())

Root Directory of Package: 
/Users/jwakim/Documents/WS_crosstalk_2024/chromo


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import chromo.mc as mc
from chromo.polymers import SSWLC
import chromo.binders
from chromo.fields import UniformDensityField
import chromo.mc.mc_controller as ctrl
from chromo.util.reproducibility import get_unique_subfolder_name
import chromo.util.poly_stat as ps

#### Specify Parameters

In [4]:
# Specify the properties of the reader protein
eps_marked = np.linspace(-4, 0, 9)
eps_unmarked = 0
all_mu = np.linspace(-6, 4, 11)

# Specify the dimensions of the polymer
num_beads = 100
bead_spacing = np.ones(num_beads-1, dtype=int) * 25.0
lp = 100.

# Define a fully marked chromatin fiber
H3K9me3_mark_pattern = np.atleast_2d(np.ones(num_beads, dtype=int) * 2).T
states = np.atleast_2d(np.zeros(num_beads, dtype=int)).T

# Specify the output directory
output_dir = "output_isotherms"

# Specify the duration of the simulation
num_snapshots = 20
mc_steps_per_snapshot = 100

#### Run Simulations

In [5]:
for eps_marked_ in eps_marked:
    print("Binding Affinity:", eps_marked_)
    for mu in all_mu:
        print("mu:", mu)
        
        # Pick a random seed
        random_seed = np.random.randint(0, 1E5)
        np.random.seed(random_seed)
    
        HP1 = chromo.binders.get_by_name('HP1')
        
        # Specify binding affinities for marked and unmarked sites
        HP1.bind_energy_mod =  eps_marked_
        HP1.bind_energy_no_mod = eps_unmarked
        
        # Specify the chemical potential
        HP1.chemical_potential = mu
        
        # Specify no self-interaction
        HP1.interaction_energy = 0.0
        
        # Make the binder collection
        binders = chromo.binders.make_binder_collection([HP1])
        
        # Instantiate the polymer
        polymer = SSWLC.gaussian_walk_polymer(
            'poly_1',
            num_beads,
            bead_spacing,
            lp=lp,
            states=states.copy(),
            binder_names=np.array(["HP1"]),
            chemical_mods=H3K9me3_mark_pattern.copy(),
            chemical_mod_names=np.array(["H3K9me3"])
        )
        
        # Define the field
        n_bins_x = 63
        n_bins_y = n_bins_x
        n_bins_z = n_bins_x
        
        x_width = 1000
        y_width = x_width
        z_width = x_width
        
        udf = UniformDensityField(
            polymers = [polymer],
            binders = binders,
            x_width = x_width,
            nx = n_bins_x,
            y_width = y_width,
            ny = n_bins_y,
            z_width = z_width,
            nz = n_bins_z,
            chi=0,
            vf_limit=1.0
        )
        
        # Specify bounds of the move
        amp_bead_bounds, amp_move_bounds = mc.get_amplitude_bounds(
            polymers = [polymer]
        )
        
        # Instantiate the move controllers
        latest_sim = get_unique_subfolder_name(f"{output_dir}/sim_")
        moves_to_use = ctrl.only_binding_move(
            log_dir=latest_sim,
            bead_amp_bounds=amp_bead_bounds.bounds,
            move_amp_bounds=amp_move_bounds.bounds,
            controller=ctrl.SimpleControl
        )
        
        # Run the simulation
        mc.polymer_in_field(
            polymers = [polymer],
            binders = binders,
            field = udf,
            num_save_mc = mc_steps_per_snapshot,
            num_saves = num_snapshots,
            bead_amp_bounds = amp_bead_bounds,
            move_amp_bounds = amp_move_bounds,
            output_dir = output_dir,
            mc_move_controllers = moves_to_use,
            random_seed=random_seed
        )

Binding Affinity: -4.0
mu: -6.0
Save point 0 completed
Save point 1 completed
Save point 2 completed
Save point 3 completed
Save point 4 completed
Save point 5 completed
Save point 6 completed
Save point 7 completed
Save point 8 completed
Save point 9 completed
Save point 10 completed
Save point 11 completed
Save point 12 completed
Save point 13 completed
Save point 14 completed
Save point 15 completed
Save point 16 completed
Save point 17 completed
Save point 18 completed
Save point 19 completed
Simulation Runtime (in seconds):  2.23
mu: -5.0
Save point 0 completed
Save point 1 completed
Save point 2 completed
Save point 3 completed
Save point 4 completed
Save point 5 completed
Save point 6 completed
Save point 7 completed
Save point 8 completed
Save point 9 completed
Save point 10 completed
Save point 11 completed
Save point 12 completed
Save point 13 completed
Save point 14 completed
Save point 15 completed
Save point 16 completed
Save point 17 completed
Save point 18 completed
Save